## Essential Imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from keras.regularizers import l2
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from tensorflow.python.keras.applications.vgg16 import preprocess_input
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

## Essential Constants

In [ ]:
train_path = '/content/drive/My Drive/DM Course Project/Temp data(plot images)/temp data(plot images)/train/'
test_path = '/content/drive/My Drive/DM Course Project/Temp data(plot images)/temp data(plot images)/test/'

# useful for getting number of classes
folders = glob('/content/drive/My Drive/DM Course Project/Temp data(plot images)/temp data(plot images)/train/*')

num_classes=len(folders)

# re-size all the images to this
IMAGE_SIZE = [224, 224]

## Creating a new model

In [ ]:
model = Sequential()
model.add(VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False))
model.add(Flatten())
model.add(Dense(units=128, activation='relu'))
model.add(Dense(1, kernel_regularizer=l2(0.01),activation='linear')) # Final Layer is SVM Layer
model.layers[0].trainable = False

58892288/58889256 [==============================] - 1s 0us/step


In [ ]:
model.compile(optimizer='adam', loss='hinge', metrics=['accuracy',tf.keras.metrics.MeanAbsoluteError(),tf.keras.metrics.MeanSquaredError(),tf.keras.metrics.Hinge(),tf.keras.metrics.BinaryCrossentropy()])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               3211392   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 17,926,209
Trainable params: 3,211,521
Non-trainable params: 14,714,688
_________________________________________________________________


## Setting up training and testing data

In [ ]:
data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)
train_set = data_generator.flow_from_directory(train_path,target_size = (224, 224),batch_size=10,class_mode = 'binary')
test_set = data_generator.flow_from_directory(test_path,target_size = (224, 224),batch_size=10,class_mode = 'binary')

Found 3000 images belonging to 2 classes.
Found 1400 images belonging to 2 classes.


In [ ]:
r = model.fit_generator(
  train_set,
  validation_data=test_set,
  epochs=20,
  steps_per_epoch=len(train_set),
  validation_steps=len(test_set)
)

f=open('/content/drive/My Drive/DM Course Project/Transfer Learning Model/TL-PI-VGG16-stats.txt','w')
f.write(str(r.history))
f.close()

model.save('/content/drive/My Drive/DM Course Project/Transfer Learning Model/TL-PI-VGG16-model.h5')

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
300/300 [==============================] - 1326s 4s/step - loss: 2.6751 - accuracy: 0.6899 - mean_absolute_error: 6.6206 - mean_squared_error: 466.9840 - hinge: 2.6573 - binary_crossentropy: 4.0123 - val_loss: 0.3014 - val_accuracy: 0.8857 - val_mean_absolute_error: 2.6237 - val_mean_squared_error: 12.0227 - val_hinge: 0.2863 - val_binary_crossentropy: 1.1950
Epoch 2/20
300/300 [==============================] - 27s 91ms/step - loss: 0.2560 - accuracy: 0.8833 - mean_absolute_error: 2.3685 - mean_squared_error: 9.0925 - hinge: 0.2415 - binary_crossentropy: 1.1997 - val_loss: 0.2851 - val_accuracy: 0.9114 - val_mean_absolute_error: 2.8160 - val_mean_squared_error: 13.5208 - val_hinge: 0.2725 - val_binary_crossentropy: 0.8950
Epoch 3/20
300/300 [==============================] - 27s 90ms/step - loss: 0.1408 - accuracy: 0.9316 - mean_absolute_error: 2.8162 - mean_squared_error: 12.9425 - hinge: 0.1286 - binary_crossentropy: 0.5994 - val_loss: 0.1661 - val_accuracy: 0.9414 - val_